# Anwendungsübungen zum RSA Algorithmus

<a target="_blank" href="https://colab.research.google.com/github/SkriptenMk/I_gW_23-27/blob/main/docs/250331/anwendung_rsa.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Aufgabe 1

Erstellen Sie ein Schlüsselpaar für die Verschlüsselung mit RSA, mit dem Sie
dreistellige Zahlen verschlüsseln können.

```{admonition} Mögliche Lösung
:class: tip, dropdown
Damit eine dreistellige Zahl verschlüsselt werden kann, muss das RSA-Modul
grösser als 1'000 sein. Es müssen also zwei Primzahlen $p$ und $q$ gefunden
werden, die etwas grösser als $\sqrt{1000}$ sind. Eine Hilfe für das Finden von
Primzahlen ist zum Beispiel die Seite
"[Die
Primzahlenseite](https://www.arndt-bruenner.de/mathe/scripts/primzahlen.htm)".

Für das Beispiel werden die Primzahlen $p = 37$ und $q = 41$ gewählt. Das RSA-Modul
ist dann $n = p \cdot q = 37 \cdot 41 = 1517$.  
Für die Verschlüsselung wird der öffentliche Exponent $e = 17$ gewählt.

Für die Berechnung des privaten Exponenten $d$ wird der erweiterte Euklidsche
Algorithmus verwendet. Dazu wird in einer ersten Phase der $ggT((p-1) \cdot
(q-1), e)$ berechnet. In einer zweiten Phase wird $s$ aus der Gleichung $ggT((p-1) \cdot
(q-1), e) = p \cdot (p-1) \cdot (q-1) + s \cdot e$ als $d$ berechnet.

Phase 1:

$$
\begin{aligned}
a&=q \cdot b + r\\
1440&=84 \cdot 17 + 12\\
17&=1 \cdot 12 + 5\\
12&=2 \cdot 5 + 2\\
2&=1 \cdot 2 + 0\\
\end{aligned}
$$

Phase 2:

$$
\begin{alignat*}{3}
&S_0 &&= 0&&\\
&S_1 &&= 1&&\\
&S_2 &&= S_0 - q_1 \cdot S_1 = 0 - 84 \cdot 1 &&= -84\\
&S_3 &&= S_1 - q_2 \cdot S_2 = 1 - 1 \cdot (-84) &&= 85\\
&S_4 &&= S_2 - q_3 \cdot S_3 = -84 - 2 \cdot 85 &&= -254\\
&S_5 &&= S_3 - q_4 \cdot S_4 = 85 - 2 \cdot (-254) &&= 593\\
\end{alignat*}
$$

Der private Exponent ist $d = 593$.
```

### Lösungen mit Hilfe von Funktionen

In [1]:
def erweiterter_euklid(a: int, b: int) -> tuple[int, int, int]:
    """
    Implementiert den erweiterten euklidischen Algorithmus zur Berechnung des GGT und der Bezout-Koeffizienten.
    
    Diese Funktion berechnet nicht nur den größten gemeinsamen Teiler (GGT) zweier Zahlen,
    sondern auch die Bezout-Koeffizienten x und y, sodass gilt: ax + by = GGT(a,b)
    
    Args:
        a (int): Erste Eingabezahl
        b (int): Zweite Eingabezahl
        
    Returns:
        tuple[int, int, int]: Ein Tupel (gcd, x, y), wobei:
            - gcd: der größte gemeinsame Teiler von a und b
            - x, y: die Bezout-Koeffizienten, für die gilt ax + by = gcd
            
    Example:
        >>> erweiterter_euklid(35, 15)
        (5, -1, 3)  # bedeutet: 35*(-1) + 15*3 = 5
    """
    # Basisfall: Wenn a gleich 0 ist, ist b der GGT
    # Die Koeffizienten sind dann 0 und 1, da: 0*a + 1*b = b
    if a == 0:
        return b, 0, 1
    
    # Rekursiver Aufruf mit dem Rest von b/a und a
    # Dies reduziert das Problem schrittweise bis zum Basisfall
    gcd, x1, y1 = erweiterter_euklid(b % a, a)
    
    # Berechnung der aktuellen Koeffizienten aus den rekursiv berechneten Werten
    # Diese Formeln ergeben sich aus der Umformung der Bezout-Identität
    x = y1 - (b // a) * x1  # x ist der neue Koeffizient für a
    y = x1                  # y ist der neue Koeffizient für b
    
    return gcd, x, y

def decryptor(a: int, b: int) -> int:
    """
    Berechnet das multiplikative Inverse von b modulo a mittels des erweiterten euklidischen Algorithmus.
    
    Diese Funktion wird in der RSA-Kryptographie verwendet, um den privaten Schlüssel d
    aus dem öffentlichen Exponenten e zu berechnen, sodass: d*e ≡ 1 (mod φ(n))
    
    Args:
        a (int): Der Modulus (in RSA: φ(n) = (p-1)(q-1))
        b (int): Die Zahl, deren multiplikatives Inverses gesucht wird (in RSA: der öffentliche Exponent e)
        
    Returns:
        int: Das positive multiplikative Inverse von b modulo a
             (in RSA: der private Exponent d)
        
    Example:
        >>> decryptor(1440, 17)
        593  # bedeutet: 17 * 593 ≡ 1 (mod 1440)
    """
    # Berechne die Koeffizienten mit dem erweiterten euklidischen Algorithmus
    # Der Koeffizient y ist das gesuchte multiplikative Inverse
    gcd, x, y = erweiterter_euklid(a, b)
    
    # Falls das berechnete Inverse negativ ist, addiere den Modulus
    # Dies garantiert ein positives Ergebnis im Bereich [0, a-1]
    if y < 0:
        return y + a
    else:
        return y

## Aufgabe 2

Codieren Sie einen einfachen Drei-Wort-Satz (z.B. "Die Katze schläft") mit Hilfe
der ASCII- bzw. Unicode-Tabelle. Sie können dabezi entweder die Unicode-Tabelle
zu Hilfe nehmen oder die Python Funktion `ord()` verwenden.  
Achten Sie darauf, dass Sie bei Buchstaben mit eienem zweistelligen ASCII-Wert
eine führende Null voranstellen.

### Semi-manuelle Lösung

In [2]:
# Semi-manuelle Lösung

print(f'D: {ord("D")}') if ord("D") > 99 else print(f'D: 0{ord("D")}')
print(f'i: {ord("i")}') if ord("i") > 99 else print(f'i: 0{ord("i")}')
print(f'e: {ord("e")}') if ord("e") > 99 else print(f'e: 0{ord("e")}')
print(f'K: {ord("K")}') if ord("K") > 99 else print(f'K: 0{ord("K")}')
print(f'a: {ord("a")}') if ord("a") > 99 else print(f'a: 0{ord("a")}')
print(f't: {ord("t")}') if ord("t") > 99 else print(f't: 0{ord("t")}')
print(f'z: {ord("z")}') if ord("z") > 99 else print(f'z: 0{ord("z")}')
print(f'e: {ord("e")}') if ord("e") > 99 else print(f'e: 0{ord("e")}')
print(f's: {ord("s")}') if ord("s") > 99 else print(f's: 0{ord("s")}')
print(f'c: {ord("c")}') if ord("c") > 99 else print(f'c: 0{ord("c")}')
print(f'h: {ord("h")}') if ord("h") > 99 else print(f'h: 0{ord("h")}')
print(f'l: {ord("l")}') if ord("l") > 99 else print(f'l: 0{ord("l")}')
print(f'ä: {ord("ä")}') if ord("ä") > 99 else print(f'ä: 0{ord("ä")}')
print(f'f: {ord("f")}') if ord("f") > 99 else print(f'f: 0{ord("f")}')
print(f't: {ord("t")}') if ord("t") > 99 else print(f't: 0{ord("t")}')
print(f'Punkt: {ord(".")}') if ord(".") > 99 else print(f'Punkt: 0{ord(".")}')

D: 068
i: 105
e: 101
K: 075
a: 097
t: 116
z: 122
e: 101
s: 115
c: 099
h: 104
l: 108
ä: 228
f: 102
t: 116
Punkt: 046


### Lösung mit Hilfe einer Funktion

In [3]:
# Lösung mit Hilfe einer Funktion

def text_codierer(text: str) -> str:
    """
    Kodiert einen Text in eine numerische Darstellung.

    Diese Funktion wandelt jeden Buchstaben des Eingabetextes in seine ASCII-
    numerische Darstellung um. Dabei wird sichergestellt, dass jede Zahl
    mindestens zweistellig ist, indem bei Bedarf führende Nullen hinzugefügt
    werden. Die resultierende Zahlenfolge wird formatiert, sodass nach jeweils
    drei Zeichen ein Leerzeichen und nach jeweils fünfzehn Zeichen ein
    Zeilenumbruch eingefügt wird.

    Args:
        text (str): Der zu kodierende Eingabetext.

    Returns:
        str: Der kodierte Text als formatierte Zahlenfolge.

    Beispiel:
        >>> text_codierer("Hallo")
        '072 097 108 108 111'
        
        >>> text_codierer("ABC123")
        '065 066 067 049 050 051'

    Hinweise:
        - Jedes Zeichen wird in seinen ASCII-Wert umgewandelt
        - Zahlen unter 100 werden mit einer führenden Null aufgefüllt
        - Alle drei Ziffern wird ein Leerzeichen eingefügt
        - Alle fünfzehn Ziffern wird ein Zeilenumbruch eingefügt
        - Führende und nachfolgende Leerzeichen werden entfernt
    """
    # Initialisiere leeren String für die ASCII-Codes
    code = ""
    
    # Wandle jeden Buchstaben in seinen ASCII-Wert um
    for char in text:
        # Ermittle den ASCII-Wert des aktuellen Zeichens
        nr = ord(char)
        # Füge eine führende Null hinzu, falls der ASCII-Wert kleiner als 100 ist
        if nr <= 99:
            code += "0"
        code += str(nr)
    
    # Initialisiere leeren String für das formatierte Ergebnis
    resultat = ""
    
    # Füge Formatierung (Leerzeichen und Zeilenumbrüche) hinzu
    for i in range(1, len(code)+1):
        if i % 3 == 0 and i != 0:
            # Füge nach jedem dritten Zeichen ein Leerzeichen ein
            resultat += code[i-1] + " "
        elif i % 15 == 0 and i != 0:
            # Füge nach jedem fünfzehnten Zeichen einen Zeilenumbruch ein
            resultat += code[i-1] + "\n"
        else:
            # Füge das Zeichen ohne zusätzliche Formatierung hinzu
            resultat += code[i-1]
    
    # Entferne überflüssige Leerzeichen am Anfang und Ende
    return resultat.strip()

# Beispielaufruf der Funktion
text = "Die Katze schläft."
print(text_codierer(text))

068 105 101 032 075 097 116 122 101 032 115 099 104 108 228 102 116 046


## Aufgabe 3

Verschlüsseln Sie den Satz aus Aufgabe 2 mit dem RSA-Algorithmus.
Verwenden Sie dazu den öffentlichen Schlüssel aus Aufgabe 1.

### Semi-manuelle Lösung

In [4]:
print(pow(68, 17, 1517), end=" ")
print(pow(105, 17, 1517), end=" ")
print(pow(101, 17, 1517), end=" ")
print(pow(32, 17, 1517), end=" ")
print(pow(75, 17, 1517), end=" ")
print(pow(97, 17, 1517), end=" ")
print(pow(116, 17, 1517), end=" ")
print(pow(122, 17, 1517), end=" ")
print(pow(101, 17, 1517), end=" ")
print(pow(32, 17, 1517), end=" ")
print(pow(115, 17, 1517), end=" ")
print(pow(99, 17, 1517), end=" ")
print(pow(104, 17, 1517), end=" ")
print(pow(108, 17, 1517), end=" ")
print(pow(228, 17, 1517), end=" ")
print(pow(102, 17, 1517), end=" ")
print(pow(116, 17, 1517), end=" ")
print(pow(46, 17, 1517))

68 660 714 237 667 637 503 286 714 237 1027 1113 1131 1085 783 115 503 144


### Lösung mit Hilfe einer Funktion

In [5]:
def rsa_anwendung(text_codiert: str, e: int, n: int) -> str:
    """
    Wendet die RSA-Verschlüsselung auf einen kodierten Text an.

    Diese Funktion führt eine RSA-Verschlüsselung auf einen vorkodierten Text durch,
    wobei jede Zahl im Eingabetext mit der RSA-Formel C = M^e mod n verschlüsselt wird.
    Die Funktion stellt sicher, dass alle ausgegebenen Zahlen mindestens zweistellig sind,
    indem bei Bedarf führende Nullen hinzugefügt werden.

    Args:
        text_codiert (str): Der zu verschlüsselnde Text in kodierter Form.
                           Format: Zahlen durch Leerzeichen getrennt (z.B. "065 066 067")
        e (int): Der öffentliche RSA-Exponent (public key component).
                Typischerweise eine kleine Primzahl wie 3 oder 65537.
        n (int): Der RSA-Modulus (public key component).
                Das Produkt zweier großer Primzahlen p und q.

    Returns:
        str: Der verschlüsselte Text als Folge von Zahlen,
             durch Leerzeichen getrennt.

    Beispiel:
        >>> rsa_anwendung("065 066", 17, 3233)
        '2790 2831'

    Hinweise:
        - Die Funktion verwendet die RSA-Verschlüsselungsformel C = M^e mod n
        - Jede Eingabezahl wird als separater Block verschlüsselt
        - Zahlen unter 100 werden mit einer führenden Null ausgegeben
        - Der Parameter e muss teilerfremd zu (p-1)(q-1) sein
        - Der Parameter n sollte ausreichend groß sein für sichere Verschlüsselung
    """
    # Liste für die verschlüsselten Werte (Chiffretexte)
    chiffres = []
    
    # Zerlege den Eingabetext in einzelne Zahlen-Strings
    triples = text_codiert.split(" ")
    
    # Verarbeite jede Zahl einzeln
    for triple in triples:
        # Konvertiere den Zahlen-String in einen Integer für die RSA-Berechnung
        t = int(triple)
        
        # Führe die RSA-Verschlüsselung durch: c = t^e mod n
        # Verwendet pow() für effiziente modulare Exponentiation
        c = pow(t, e, n)
        
        # Stelle sicher, dass jede verschlüsselte Zahl drei Stellen hat
        if c < 100:
            chiffres.append("0" + str(c))  # Füge führende Null hinzu
        else:
            chiffres.append(str(c))
    
    # Verbinde alle verschlüsselten Zahlen mit Leerzeichen zu einem String
    return " ".join(chiffres)

code = "068 105 101 032 075 097 116 122 101 032 115 099 104 108 228 102 116 046"
chiffrat = rsa_anwendung(code, 17, 1517)
print(chiffrat)  

068 660 714 237 667 637 503 286 714 237 1027 1113 1131 1085 783 115 503 144


## Aufgabe 4

Testen Sie Ihr Ergebnis, indem Sie den verschlüsselten Text mit dem privaten Schlüssel
aus Aufgabe 1 entschlüsseln.

### Semi-manuelle Lösung

In [6]:
print(f'068: {pow(68, 593, 1517)} -> {chr(pow(68, 593, 1517))}')
print(f'660: {pow(660, 593, 1517)} -> {chr(pow(660, 593, 1517))}')
print(f'714: {pow(714, 593, 1517)} -> {chr(pow(714, 593, 1517))}')
print(f'237: {pow(237, 593, 1517)} -> {chr(pow(237, 593, 1517))}')
print(f'667: {pow(667, 593, 1517)} -> {chr(pow(667, 593, 1517))}')
print(f'637: {pow(637, 593, 1517)} -> {chr(pow(637, 593, 1517))}')
print(f'503: {pow(503, 593, 1517)} -> {chr(pow(503, 593, 1517))}')
print(f'286: {pow(286, 593, 1517)} -> {chr(pow(286, 593, 1517))}')
print(f'714: {pow(714, 593, 1517)} -> {chr(pow(714, 593, 1517))}')
print(f'237: {pow(237, 593, 1517)} -> {chr(pow(237, 593, 1517))}')
print(f'1027: {pow(1027, 593, 1517)} -> {chr(pow(1027, 593, 1517))}')
print(f'1113: {pow(1113, 593, 1517)} -> {chr(pow(1113, 593, 1517))}')
print(f'1113: {pow(1113, 593, 1517)} -> {chr(pow(1113, 593, 1517))}')
print(f'1085: {pow(1085, 593, 1517)} -> {chr(pow(1085, 593, 1517))}')
print(f'783: {pow(783, 593, 1517)} -> {chr(pow(783, 593, 1517))}')
print(f'115: {pow(115, 593, 1517)} -> {chr(pow(115, 593, 1517))}')
print(f'503: {pow(503, 593, 1517)} -> {chr(pow(503, 593, 1517))}')
print(f'144: {pow(144, 593, 1517)} -> {chr(pow(144, 593, 1517))}')

068: 68 -> D
660: 105 -> i
714: 101 -> e
237: 32 ->  
667: 75 -> K
637: 97 -> a
503: 116 -> t
286: 122 -> z
714: 101 -> e
237: 32 ->  
1027: 115 -> s
1113: 99 -> c
1113: 99 -> c
1085: 108 -> l
783: 228 -> ä
115: 102 -> f
503: 116 -> t
144: 46 -> .


### Lösung mit Hilfe einer Funktion

In [7]:
chiffrat ='068 660 714 237 667 637 503 286 714 237 1027 1113 1131 1085 783 115 503 144'

text = rsa_anwendung(chiffrat, 593, 1517)
print(text)


068 105 101 032 075 097 116 122 101 032 115 099 104 108 228 102 116 046
